In [1]:
%pip install comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.9/561.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.2
    Uninstalling websocket-client-1.6.2:
      Successfully uninstalled websocket-client-1.6.2
  Attempting uninstall: python-box
    Found existing installation: python-box 7.1.1
    Uninstalling python-box-7.1.1:
      Successfully uninstalled python-box-7.1.1


In [10]:
import comet_ml
comet_ml.init(project_name="Prueba comet")

In [11]:
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint
import gzip
import pickle
import numpy as np

In [13]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/eduardo-macavity/prueba-comet/7d457e17d2bd4f8e807465177f272b48



In [8]:
learning_rate = 0.001
epochs = 30
batch_size = 120

In [25]:
modelo_cargado = tf.keras.models.load_model('best_model.hdf5')

In [32]:
parameters = {
    "batch_size": 128,
    "epochs": 30,
    "optimizer": "adam",
    "loss": "binary_crossentropy",
}

experiment.log_parameters(parameters)

In [15]:
#Cargamos los datos
f = gzip.open('mnist.pkl.gz', 'rb')
dataset = pickle.load(f,encoding="latin1")

In [16]:
dat=np.array(dataset)
print(dat[1,1].shape)
(x_train, y_train), (x_val, y_val), (x_test, y_test) = dataset

(10000,)


<ipython-input-16-9d04016ce3ca>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dat=np.array(dataset)


In [17]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [18]:
num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [33]:
model = Sequential(modelo_cargado)
model.add(Dense(512, activation='sigmoid', input_shape=(784,)))
model.add(Dense(600,activation='tanh'))
model.add(Dense(200,activation='selu'))
model.add(Dense(600,activation='relu'))
model.add(Dense(100,activation='elu'))
model.add(Dense(400, activation='selu', kernel_regularizer=regularizers.L1(0.01) )) #Regularizacion L1
model.add(Dense(600,activation='sigmoid'))
model.add(Dense(500,activation='exponential'))
model.add(Dense(num_classes, activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 10)                1376730   
                                                                 
 dense_25 (Dense)            (None, 512)               5632      
                                                                 
 dense_26 (Dense)            (None, 600)               307800    
                                                                 
 dense_27 (Dense)            (None, 200)               120200    
                                                                 
 dense_28 (Dense)            (None, 600)               120600    
                                                                 
 dense_29 (Dense)            (None, 100)               60100     
                                                                 
 dense_30 (Dense)            (None, 400)              

In [20]:
# specify the path where you want to save the model
#filepath = "best_model.hdf5"

# initialize the ModelCheckpoint callback
#checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [34]:
model.compile(loss=parameters['loss'],optimizer=parameters['optimizer'],metrics=['accuracy'])


In [35]:
model.fit(x_train, y_trainc,
                    batch_size=parameters['batch_size'],
                    epochs=parameters["epochs"],
                    verbose=1,
                    validation_data=(x_test, y_testc),
                    )

Epoch 1/30
417/417 [==============================] - 35s 76ms/step - loss: 2.3568 - accuracy: 0.1053 - val_loss: 0.3722 - val_accuracy: 0.0958
Epoch 2/30
417/417 [==============================] - 31s 75ms/step - loss: 0.3714 - accuracy: 0.1046 - val_loss: 0.3697 - val_accuracy: 0.0982
Epoch 3/30
417/417 [==============================] - 35s 83ms/step - loss: 0.3701 - accuracy: 0.1048 - val_loss: 0.3697 - val_accuracy: 0.1135
Epoch 4/30
417/417 [==============================] - 32s 77ms/step - loss: 0.3690 - accuracy: 0.1080 - val_loss: 0.3690 - val_accuracy: 0.1135
Epoch 5/30
417/417 [==============================] - 31s 74ms/step - loss: 0.3683 - accuracy: 0.1056 - val_loss: 0.3680 - val_accuracy: 0.1135
Epoch 6/30
417/417 [==============================] - 33s 78ms/step - loss: 0.3675 - accuracy: 0.1076 - val_loss: 0.3676 - val_accuracy: 0.1028
Epoch 7/30
417/417 [==============================] - 34s 82ms/step - loss: 0.3674 - accuracy: 0.1086 - val_loss: 0.3670 - val_accuracy:

In [36]:
score = model.evaluate(x_test, y_testc, verbose=1) #evaluar la eficiencia del modelo
print(score)
a=model.predict(x_test) #predicción de la red entrenada
print(a.shape)
print(a[1])
print("resultado correcto:")
print(y_testc[1])


313/313 [==============================] - 4s 13ms/step - loss: 0.3669 - accuracy: 0.1032
[0.36685875058174133, 0.10320000350475311]
313/313 [==============================] - 4s 13ms/step
(10000, 10)
[0.0935039  0.10666244 0.10771217 0.10760037 0.09827911 0.08267973
 0.10107709 0.10611781 0.09358405 0.09770516]
resultado correcto:
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [37]:
experiment.log_model("MNIST1", "best_model.hdf5")

{'web': 'https://www.comet.com/api/asset/download?assetId=be8e14c22959445bb5c58860f3c9e553&experimentKey=7d457e17d2bd4f8e807465177f272b48',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=be8e14c22959445bb5c58860f3c9e553&experimentKey=7d457e17d2bd4f8e807465177f272b48',
 'assetId': 'be8e14c22959445bb5c58860f3c9e553'}

In [38]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/eduardo-macavity/prueba-comet/7d457e17d2bd4f8e807465177f272b48
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [97]                 : (0.10288000106811523, 0.9986799955368042)
COMET INFO:     batch_accuracy [3980]         : (0.0390625, 1.0)
COMET INFO:     batch_loss [3980]             : (1.5389645341201685e-05, 22.718931198120117)
COMET INFO:     epoch_duration [97]           : (15.976230362999786, 42.51298712300013)
COMET INFO:     loss [97]                     : (0.0008520971750840545, 2.4865736961364746)
COMET INFO:     val_accuracy [97]             : (0.0957999974489212, 0.9818999767303467)
COMET INFO:   